In [9]:
import numpy as np
import matplotlib.pyplot as plt

In [10]:
pip install PyQt5


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\sheet\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [11]:
%matplotlib qt

In [12]:

#-------Flip-Flop function-------
def flip_flop(vel_int):
    return 1 if vel_int >= 0 else -1

#--------Flux Limiter Function--------
def flux_int(vel_int,ql_2,ql_1,q_0,dt,dx,limiter):
    
    theta = flip_flop(vel_int)
    r = (ql_1 - ql_2)/(q_0 - ql_1)
    
    if limiter == "Donor Cell":
        
        term1 = 0.5*vel_int*(ql_1*(1+theta) + q_0*(1-theta))
        term2 = 0.5*abs(vel_int)*(1-abs(vel_int*dt/dx))*0.0*(q_0-ql_1)

        flux_val = term1 + term2

        return flux_val
    
    elif limiter == "Lax Wendroff":
        
        term1 = 0.5*vel_int*(ql_1*(1+theta) + q_0*(1-theta))
        term2 = 0.5*abs(vel_int)*(1-abs(vel_int*dt/dx))*1.0*(q_0-ql_1)

        flux_val = term1 + term2

        return flux_val
    
    elif limiter == "Beam Warming":
        
        term1 = 0.5*vel_int*(ql_1*(1+theta) + q_0*(1-theta))
        if q_0 == ql_1: 
            term2 = 0
        else:
            term2 = 0.5*abs(vel_int)*(1-abs(vel_int*dt/dx))*r*(q_0-ql_1)

        flux_val = term1 + term2

        return flux_val
    
    elif limiter == "Fromm":
        
        term1 = 0.5*vel_int*(ql_1*(1+theta) + q_0*(1-theta))
        if q_0 == ql_1:
            term2 = 0
        else:
            term2 = 0.5*abs(vel_int)*(1-abs(vel_int*dt/dx))*0.5*(1+r)*(q_0-ql_1)

        flux_val = term1 + term2

        return flux_val
    
    elif limiter == "minmod":
        
        term1 = 0.5*vel_int*(ql_1*(1+theta) + q_0*(1-theta))
        if q_0 == ql_1:
            term2 = 0
        else:
            minmod = max(0, min(1,r))
            term2 = 0.5*abs(vel_int)*(1-abs(vel_int*dt/dx))*minmod*(q_0 - ql_1)
        
        flux_val = term1 + term2
        
        return flux_val
    
    elif limiter == "superbee":
        
        term1 = 0.5*vel_int*(ql_1*(1+theta) + q_0*(1-theta))
        if q_0 == ql_1:
            term2 = 0
        else:
            superbee = max(0, min(1, 2*r), min(2, r))        
            term2 = 0.5*abs(vel_int)*(1-abs(vel_int*dt/dx))*superbee*(q_0 - ql_1)
        
        flux_val = term1 + term2
        
        return flux_val
    
    elif limiter == "mc":
        
        term1 = 0.5*vel_int*(ql_1*(1+theta) + q_0*(1-theta))
        if q_0 == ql_1:
            term2 = 0
        else:
            mc = max(0, min(0.5*(1+r), 2, 2*(r)))
            term2 = 0.5*abs(vel_int)*(1-abs(vel_int*dt/dx))*mc*(q_0-ql_1)
        
        flux_val = term1 + term2
        
        return flux_val
        
    elif limiter == "vanleer":
        
        term1 = 0.5*vel_int*(ql_1*(1+theta) + q_0*(1-theta))
        if q_0 == ql_1:
            term2 = 0
        else:
            vanleer1 = r + abs(r)
            vanleer2 = 1 + abs(r)
            vanleer = vanleer1/vanleer2
            
            term2 = 0.5*abs(vel_int)*(1-abs(vel_int*dt/dx))*vanleer*(q_0 - ql_1)
        
        flux_val = term1 + term2
        
        return flux_val
    
#------------Pulse function---------
def pulse(t,x,problem):
    sigma = 1.0
    q = np.zeros([len(t),len(x)])
    
    if problem == "Square Pulse": 
        
        for i in range(len(x)):
            q[0][i] = 0.00001 if x[i] < -2 or x[i] > 2 else 1
        
        return q

    elif problem == "Gaussian Pulse":    
        
        q[0,:] = (1/(sigma*np.sqrt(2*np.pi)))*np.exp(-(x**2)/(2*(sigma**2)))
        
        return q    

In [13]:
#--------Initial Condition-------
x_vals = np.linspace(-10,10,200,endpoint=True)
t_vals = np.linspace(0,10,1000,endpoint=True)

adv_vel = 3.0
sigma = 1.0
pi = np.pi

dt = t_vals[1] - t_vals[0]
dx = x_vals[1] - x_vals[0]

print(dt/dx)

#-----------Gaussian wave------
problem = "Gaussian Pulse"
q_vals = pulse(t_vals,x_vals,problem)



0.0995995995995996


In [14]:
#----------Donor Cell-------
limiter = "Donor Cell"

fig, ax = plt.subplots(figsize=(14,8),dpi=80)
res, = ax.plot(x_vals,q_vals[0],lw=3.0,ls='-',color='r')
plt.xticks(size=15)
plt.yticks(size=15)
plt.xlabel("Increasing $x$ values $\longrightarrow$",size=18)
plt.ylabel("Increasing $q$ Values",size=18)
plt.title(limiter,size=20,fontweight='bold')
text = ax.text(8.5,0.97,r'Time: 0.00',fontsize=15,bbox={'facecolor':'pink','alpha':0.8,'pad':16})
plt.pause(2)

for n in range(len(t_vals)-1):
    for i in range(1,len(x_vals)-1):
        
        flux_left = flux_int(adv_vel,q_vals[n,i-2],q_vals[n,i-1],q_vals[n,i],dt,dx,limiter)
        flux_right = flux_int(adv_vel,q_vals[n,i-1],q_vals[n,i],q_vals[n,i+1],dt,dx,limiter)
        
        df = flux_right - flux_left
        
        q_vals[n+1,i] = q_vals[n,i] - (dt/dx)*df
        
    q_vals[n+1,0] = q_vals[n,-2]
    q_vals[n+1,-1] = q_vals[n,1]
        
    res.set_ydata(q_vals[n+1])
    text.set_text(r'Time: %.2f'%t_vals[n])
    fig.canvas.draw()
    fig.canvas.flush_events()
    
q_vals_donor = q_vals 

In [15]:
#------Lax Wandroff------
limiter = "Lax Wendroff"

fig, ax = plt.subplots(figsize=(14,8),dpi=80)
res, = ax.plot(x_vals,q_vals[0],lw=3.0,ls='-',color='r')
plt.xticks(size=15)
plt.yticks(size=15)
plt.xlabel("Increasing $x$ values $\longrightarrow$",size=18)
plt.ylabel("Increasing $q$ Values",size=18)
plt.title(limiter,size=20,fontweight='bold')
text = ax.text(8.5,0.97,r'Time: 0.00',fontsize=15,bbox={'facecolor':'pink','alpha':0.8,'pad':16})
plt.pause(2)

for n in range(len(t_vals)-1):
    for i in range(1,len(x_vals)-1):
        
        flux_left = flux_int(adv_vel,q_vals[n,i-2],q_vals[n,i-1],q_vals[n,i],dt,dx,limiter)
        flux_right = flux_int(adv_vel,q_vals[n,i-1],q_vals[n,i],q_vals[n,i+1],dt,dx,limiter)
        
        df = flux_right - flux_left
        
        q_vals[n+1,i] = q_vals[n,i] - (dt/dx)*df
        
    q_vals[n+1,0] = q_vals[n,-2]
    q_vals[n+1,-1] = q_vals[n,1]
        
    res.set_ydata(q_vals[n+1])
    text.set_text(r'Time: %.2f'%t_vals[n])
    fig.canvas.draw()
    fig.canvas.flush_events()
    
q_vals_laxwend = q_vals

In [16]:
#---------Beam warming----------
limiter = "Beam Warming"

fig, ax = plt.subplots(figsize=(14,8),dpi=80)
res, = ax.plot(x_vals,q_vals[0],lw=3.0,ls='-',color='r')
plt.xticks(size=15)
plt.yticks(size=15)
plt.xlabel("Increasing $x$ values $\longrightarrow$",size=18)
plt.ylabel("Increasing $q$ Values",size=18)
plt.title(limiter,size=20,fontweight='bold')
text = ax.text(8.5,0.37,r'Time: 0.00',fontsize=15,bbox={'facecolor':'pink','alpha':0.8,'pad':16})
plt.pause(2)

for n in range(len(t_vals)-1):
    for i in range(1,len(x_vals)-1):
        
        flux_left = flux_int(adv_vel,q_vals[n,i-2],q_vals[n,i-1],q_vals[n,i],dt,dx,limiter)
        flux_right = flux_int(adv_vel,q_vals[n,i-1],q_vals[n,i],q_vals[n,i+1],dt,dx,limiter)
        
        df = flux_right - flux_left
        
        q_vals[n+1,i] = q_vals[n,i] - (dt/dx)*df
        
    q_vals[n+1,0] = q_vals[n,-2]
    q_vals[n+1,-1] = q_vals[n,1]
        
    res.set_ydata(q_vals[n+1])
    text.set_text(r'Time: %.2f'%t_vals[n])
    fig.canvas.draw()
    fig.canvas.flush_events()
    
q_vals_beamwarm = q_vals

In [17]:
#---------Fromm------

limiter = "Fromm"

fig, ax = plt.subplots(figsize=(14,8),dpi=80)
res, = ax.plot(x_vals,q_vals[0],lw=3.0,ls='-',color='r')
plt.xticks(size=15)
plt.yticks(size=15)
plt.xlabel("Increasing $x$ values $\longrightarrow$",size=18)
plt.ylabel("Increasing $q$ Values",size=18)
plt.title(limiter,size=20,fontweight='bold')
text = ax.text(8.5,0.37,r'Time: 0.00',fontsize=15,bbox={'facecolor':'pink','alpha':0.8,'pad':16})
plt.pause(2)

for n in range(len(t_vals)-1):
    for i in range(1,len(x_vals)-1):
        
        flux_left = flux_int(adv_vel,q_vals[n,i-2],q_vals[n,i-1],q_vals[n,i],dt,dx,limiter)
        flux_right = flux_int(adv_vel,q_vals[n,i-1],q_vals[n,i],q_vals[n,i+1],dt,dx,limiter)
        
        df = flux_right - flux_left
        
        q_vals[n+1,i] = q_vals[n,i] - (dt/dx)*df
        
    q_vals[n+1,0] = q_vals[n,-2]
    q_vals[n+1,-1] = q_vals[n,1]
        
    res.set_ydata(q_vals[n+1])
    text.set_text(r'Time: %.2f'%t_vals[n])
    fig.canvas.draw()
    fig.canvas.flush_events()
    
q_vals_fromm = q_vals

In [18]:
#-----------Min Mod-------
limiter = "minmod"

fig, ax = plt.subplots(figsize=(14,8),dpi=80)
res, = ax.plot(x_vals,q_vals[0],lw=3.0,ls='-',color='r')
plt.xticks(size=15)
plt.yticks(size=15)
plt.xlabel("Increasing $x$ values $\longrightarrow$",size=18)
plt.ylabel("Increasing $q$ Values",size=18)
plt.title(limiter,size=20,fontweight='bold')
text = ax.text(8.5,0.97,r'Time: 0.00',fontsize=15,bbox={'facecolor':'pink','alpha':0.8,'pad':16})
plt.pause(2)

for n in range(len(t_vals)-1):
    for i in range(1,len(x_vals)-1):
        
        flux_left = flux_int(adv_vel,q_vals[n,i-2],q_vals[n,i-1],q_vals[n,i],dt,dx,limiter)
        flux_right = flux_int(adv_vel,q_vals[n,i-1],q_vals[n,i],q_vals[n,i+1],dt,dx,limiter)
        
        df = flux_right - flux_left
        
        q_vals[n+1,i] = q_vals[n,i] - (dt/dx)*df
        
    q_vals[n+1,0] = q_vals[n,-2]
    q_vals[n+1,-1] = q_vals[n,1]
        
    res.set_ydata(q_vals[n+1])
    text.set_text(r'Time: %.2f'%t_vals[n])
    fig.canvas.draw()
    fig.canvas.flush_events()
    
q_vals_minmod = q_vals

In [19]:
#-----------Superbee-------
limiter = "superbee"

fig, ax = plt.subplots(figsize=(14,8),dpi=80)
res, = ax.plot(x_vals,q_vals[0],lw=3.0,ls='-',color='r')
plt.xticks(size=15)
plt.yticks(size=15)
plt.xlabel("Increasing $x$ values $\longrightarrow$",size=18)
plt.ylabel("Increasing $q$ Values",size=18)
plt.title(limiter,size=20,fontweight='bold')
text = ax.text(8.5,0.97,r'Time: 0.00',fontsize=15,bbox={'facecolor':'pink','alpha':0.8,'pad':16})
plt.pause(2)

for n in range(len(t_vals)-1):
    for i in range(1,len(x_vals)-1):
        
        flux_left = flux_int(adv_vel,q_vals[n,i-2],q_vals[n,i-1],q_vals[n,i],dt,dx,limiter)
        flux_right = flux_int(adv_vel,q_vals[n,i-1],q_vals[n,i],q_vals[n,i+1],dt,dx,limiter)
        
        df = flux_right - flux_left
        
        q_vals[n+1,i] = q_vals[n,i] - (dt/dx)*df
        
    q_vals[n+1,0] = q_vals[n,-2]
    q_vals[n+1,-1] = q_vals[n,1]
        
    res.set_ydata(q_vals[n+1])
    text.set_text(r'Time: %.2f'%t_vals[n])
    fig.canvas.draw()
    fig.canvas.flush_events()
    
q_vals_superbee = q_vals

C:\Users\sheet\AppData\Local\Temp\ipykernel_10596\1902365474.py:9: RuntimeWarning: divide by zero encountered in scalar divide
  r = (ql_1 - ql_2)/(q_0 - ql_1)
C:\Users\sheet\AppData\Local\Temp\ipykernel_10596\1902365474.py:9: RuntimeWarning: invalid value encountered in scalar divide
  r = (ql_1 - ql_2)/(q_0 - ql_1)


In [20]:
#-----------Mc--------
limiter = "mc"

fig, ax = plt.subplots(figsize=(14,8),dpi=80)
res, = ax.plot(x_vals,q_vals[0],lw=3.0,ls='-',color='r')
plt.xticks(size=15)
plt.yticks(size=15)
plt.xlabel("Increasing $x$ values $\longrightarrow$",size=18)
plt.ylabel("Increasing $q$ Values",size=18)
plt.title(limiter,size=20,fontweight='bold')
text = ax.text(8.5,0.97,r'Time: 0.00',fontsize=15,bbox={'facecolor':'pink','alpha':0.8,'pad':16})
plt.pause(2)

for n in range(len(t_vals)-1):
    for i in range(1,len(x_vals)-1):
        
        flux_left = flux_int(adv_vel,q_vals[n,i-2],q_vals[n,i-1],q_vals[n,i],dt,dx,limiter)
        flux_right = flux_int(adv_vel,q_vals[n,i-1],q_vals[n,i],q_vals[n,i+1],dt,dx,limiter)
        
        df = flux_right - flux_left
        
        q_vals[n+1,i] = q_vals[n,i] - (dt/dx)*df
        
    q_vals[n+1,0] = q_vals[n,-2]
    q_vals[n+1,-1] = q_vals[n,1]
        
    res.set_ydata(q_vals[n+1])
    text.set_text(r'Time: %.2f'%t_vals[n])
    fig.canvas.draw()
    fig.canvas.flush_events()
    
q_vals_mc = q_vals

C:\Users\sheet\AppData\Local\Temp\ipykernel_10596\1902365474.py:9: RuntimeWarning: divide by zero encountered in scalar divide
  r = (ql_1 - ql_2)/(q_0 - ql_1)
C:\Users\sheet\AppData\Local\Temp\ipykernel_10596\1902365474.py:9: RuntimeWarning: invalid value encountered in scalar divide
  r = (ql_1 - ql_2)/(q_0 - ql_1)


In [21]:
#-------Van Leer--------
limiter = "vanleer"

fig, ax = plt.subplots(figsize=(14,8),dpi=80)
res, = ax.plot(x_vals,q_vals[0],lw=3.0,ls='-',color='r')
plt.xticks(size=15)
plt.yticks(size=15)
plt.xlabel("Increasing $x$ values $\longrightarrow$",size=18)
plt.ylabel("Increasing $q$ Values",size=18)
plt.title(limiter,size=20,fontweight='bold')
text = ax.text(8.5,0.37,r'Time: 0.00',fontsize=15,bbox={'facecolor':'pink','alpha':0.8,'pad':16})
plt.pause(2)

for n in range(len(t_vals)-1):
    for i in range(1,len(x_vals)-1):
        
        flux_left = flux_int(adv_vel,q_vals[n,i-2],q_vals[n,i-1],q_vals[n,i],dt,dx,limiter)
        flux_right = flux_int(adv_vel,q_vals[n,i-1],q_vals[n,i],q_vals[n,i+1],dt,dx,limiter)
        
        df = flux_right - flux_left
        
        q_vals[n+1,i] = q_vals[n,i] - (dt/dx)*df
        
    q_vals[n+1,0] = q_vals[n,-2]
    q_vals[n+1,-1] = q_vals[n,1]
        
    res.set_ydata(q_vals[n+1])
    text.set_text(r'Time: %.2f'%t_vals[n])
    fig.canvas.draw()
    fig.canvas.flush_events()

q_vals_vanleer = q_vals

In [22]:
# Square Wave-----
problem_square = "Square Pulse"
q_vals_square = pulse(t_vals,x_vals,problem_square)


#------Donor Cell------

limiter = "Donor Cell"

fig, ax = plt.subplots(figsize=(14,8),dpi=80)
res, = ax.plot(x_vals,q_vals_square[0],lw=3.0,ls='-',color='r')
plt.xticks(size=15)
plt.yticks(size=15)
plt.xlabel("Increasing $x$ values $\longrightarrow$",size=18)
plt.ylabel("Increasing $q$ Values",size=18)
plt.title(limiter,size=20,fontweight='bold')
text = ax.text(8.5,0.97,r'Time: 0.00',fontsize=15,bbox={'facecolor':'pink','alpha':0.8,'pad':16})
plt.pause(2)

for n in range(len(t_vals)-1):
    for i in range(1,len(x_vals)-1):
        
        flux_left = flux_int(adv_vel,q_vals_square[n,i-2],q_vals_square[n,i-1],q_vals_square[n,i],dt,dx,limiter)
        flux_right = flux_int(adv_vel,q_vals_square[n,i-1],q_vals_square[n,i],q_vals_square[n,i+1],dt,dx,limiter)
        
        df = flux_right - flux_left
        
        q_vals_square[n+1,i] = q_vals_square[n,i] - (dt/dx)*df
        
    q_vals_square[n+1,0] = q_vals_square[n,-2]
    q_vals_square[n+1,-1] = q_vals_square[n,1]
        
    res.set_ydata(q_vals_square[n+1])
    text.set_text(r'Time: %.2f'%t_vals[n])
    fig.canvas.draw()
    fig.canvas.flush_events()
    
q_vals_square_donor = q_vals_square

C:\Users\sheet\AppData\Local\Temp\ipykernel_10596\1902365474.py:9: RuntimeWarning: invalid value encountered in scalar divide
  r = (ql_1 - ql_2)/(q_0 - ql_1)
C:\Users\sheet\AppData\Local\Temp\ipykernel_10596\1902365474.py:9: RuntimeWarning: divide by zero encountered in scalar divide
  r = (ql_1 - ql_2)/(q_0 - ql_1)


In [23]:
#------Lax wendroff------
limiter = "Lax Wendroff"

fig, ax = plt.subplots(figsize=(14,8),dpi=80)
res, = ax.plot(x_vals,q_vals_square[0],lw=3.0,ls='-',color='r')
plt.xticks(size=15)
plt.yticks(size=15)
plt.xlabel("Increasing $x$ values $\longrightarrow$",size=18)
plt.ylabel("Increasing $q$ Values",size=18)
plt.title(limiter,size=20,fontweight='bold')
text = ax.text(8.5,0.97,r'Time: 0.00',fontsize=15,bbox={'facecolor':'pink','alpha':0.8,'pad':16})
plt.pause(2)

for n in range(len(t_vals)-1):
    for i in range(1,len(x_vals)-1):
        
        flux_left = flux_int(adv_vel,q_vals_square[n,i-2],q_vals_square[n,i-1],q_vals_square[n,i],dt,dx,limiter)
        flux_right = flux_int(adv_vel,q_vals_square[n,i-1],q_vals_square[n,i],q_vals_square[n,i+1],dt,dx,limiter)
        
        df = flux_right - flux_left
        
        q_vals_square[n+1,i] = q_vals_square[n,i] - (dt/dx)*df
        
    q_vals_square[n+1,0] = q_vals_square[n,-2]
    q_vals_square[n+1,-1] = q_vals_square[n,1]
        
    res.set_ydata(q_vals_square[n+1])
    text.set_text(r'Time: %.2f'%t_vals[n])
    fig.canvas.draw()
    fig.canvas.flush_events()
    
q_vals_square_laxwend = q_vals_square

C:\Users\sheet\AppData\Local\Temp\ipykernel_10596\1902365474.py:9: RuntimeWarning: invalid value encountered in scalar divide
  r = (ql_1 - ql_2)/(q_0 - ql_1)
C:\Users\sheet\AppData\Local\Temp\ipykernel_10596\1902365474.py:9: RuntimeWarning: divide by zero encountered in scalar divide
  r = (ql_1 - ql_2)/(q_0 - ql_1)


In [24]:
#------Beam Warming -----

limiter = "Beam Warming"

fig, ax = plt.subplots(figsize=(14,8),dpi=80)
res, = ax.plot(x_vals,q_vals_square[0],lw=3.0,ls='-',color='r')
plt.xticks(size=15)
plt.yticks(size=15)
plt.xlabel("Increasing $x$ values $\longrightarrow$",size=18)
plt.ylabel("Increasing $q$ Values",size=18)
plt.title(limiter,size=20,fontweight='bold')
text = ax.text(8.5,0.37,r'Time: 0.00',fontsize=15,bbox={'facecolor':'pink','alpha':0.8,'pad':16})
plt.pause(2)

for n in range(len(t_vals)-1):
    for i in range(1,len(x_vals)-1):
        
        flux_left = flux_int(adv_vel,q_vals_square[n,i-2],q_vals_square[n,i-1],q_vals_square[n,i],dt,dx,limiter)
        flux_right = flux_int(adv_vel,q_vals_square[n,i-1],q_vals_square[n,i],q_vals_square[n,i+1],dt,dx,limiter)
        
        df = flux_right - flux_left
        
        q_vals_square[n+1,i] = q_vals_square[n,i] - (dt/dx)*df
        
    q_vals_square[n+1,0] = q_vals_square[n,-2]
    q_vals_square[n+1,-1] = q_vals_square[n,1]
        
    res.set_ydata(q_vals_square[n+1])
    text.set_text(r'Time: %.2f'%t_vals[n])
    fig.canvas.draw()
    fig.canvas.flush_events()
    
q_vals_square_beamwarm = q_vals_square

C:\Users\sheet\AppData\Local\Temp\ipykernel_10596\1902365474.py:9: RuntimeWarning: invalid value encountered in scalar divide
  r = (ql_1 - ql_2)/(q_0 - ql_1)
C:\Users\sheet\AppData\Local\Temp\ipykernel_10596\1902365474.py:9: RuntimeWarning: divide by zero encountered in scalar divide
  r = (ql_1 - ql_2)/(q_0 - ql_1)


In [25]:
#------fromm-----
limiter = "Fromm"

fig, ax = plt.subplots(figsize=(14,8),dpi=80)
res, = ax.plot(x_vals,q_vals_square[0],lw=3.0,ls='-',color='r')
plt.xticks(size=15)
plt.yticks(size=15)
plt.xlabel("Increasing $x$ values $\longrightarrow$",size=18)
plt.ylabel("Increasing $q$ Values",size=18)
plt.title(limiter,size=20,fontweight='bold')
text = ax.text(8.5,0.37,r'Time: 0.00',fontsize=15,bbox={'facecolor':'pink','alpha':0.8,'pad':16})
plt.pause(2)

for n in range(len(t_vals)-1):
    for i in range(1,len(x_vals)-1):
        
        flux_left = flux_int(adv_vel,q_vals_square[n,i-2],q_vals_square[n,i-1],q_vals_square[n,i],dt,dx,limiter)
        flux_right = flux_int(adv_vel,q_vals_square[n,i-1],q_vals_square[n,i],q_vals_square[n,i+1],dt,dx,limiter)
        
        df = flux_right - flux_left
        
        q_vals_square[n+1,i] = q_vals_square[n,i] - (dt/dx)*df
        
    q_vals_square[n+1,0] = q_vals_square[n,-2]
    q_vals_square[n+1,-1] = q_vals_square[n,1]
        
    res.set_ydata(q_vals_square[n+1])
    text.set_text(r'Time: %.2f'%t_vals[n])
    fig.canvas.draw()
    fig.canvas.flush_events()
    
q_vals_square_fromm = q_vals_square

C:\Users\sheet\AppData\Local\Temp\ipykernel_10596\1902365474.py:9: RuntimeWarning: invalid value encountered in scalar divide
  r = (ql_1 - ql_2)/(q_0 - ql_1)
C:\Users\sheet\AppData\Local\Temp\ipykernel_10596\1902365474.py:9: RuntimeWarning: divide by zero encountered in scalar divide
  r = (ql_1 - ql_2)/(q_0 - ql_1)


In [26]:
#-------Min Mod-----

limiter = "minmod"

fig, ax = plt.subplots(figsize=(14,8),dpi=80)
res, = ax.plot(x_vals,q_vals_square[0],lw=3.0,ls='-',color='r')
plt.xticks(size=15)
plt.yticks(size=15)
plt.xlabel("Increasing $x$ values $\longrightarrow$",size=18)
plt.ylabel("Increasing $q$ Values",size=18)
plt.title(limiter,size=20,fontweight='bold')
text = ax.text(8.5,0.97,r'Time: 0.00',fontsize=15,bbox={'facecolor':'pink','alpha':0.8,'pad':16})
plt.pause(2)

for n in range(len(t_vals)-1):
    for i in range(1,len(x_vals)-1):
        
        flux_left = flux_int(adv_vel,q_vals_square[n,i-2],q_vals_square[n,i-1],q_vals_square[n,i],dt,dx,limiter)
        flux_right = flux_int(adv_vel,q_vals_square[n,i-1],q_vals_square[n,i],q_vals_square[n,i+1],dt,dx,limiter)
        
        df = flux_right - flux_left
        
        q_vals_square[n+1,i] = q_vals_square[n,i] - (dt/dx)*df
        
    q_vals_square[n+1,0] = q_vals_square[n,-2]
    q_vals_square[n+1,-1] = q_vals_square[n,1]
        
    res.set_ydata(q_vals_square[n+1])
    text.set_text(r'Time: %.2f'%t_vals[n])
    fig.canvas.draw()
    fig.canvas.flush_events()
    
q_vals_square_minmod = q_vals_square

C:\Users\sheet\AppData\Local\Temp\ipykernel_10596\1902365474.py:9: RuntimeWarning: invalid value encountered in scalar divide
  r = (ql_1 - ql_2)/(q_0 - ql_1)
C:\Users\sheet\AppData\Local\Temp\ipykernel_10596\1902365474.py:9: RuntimeWarning: divide by zero encountered in scalar divide
  r = (ql_1 - ql_2)/(q_0 - ql_1)


In [27]:
#--------Superbee------
limiter = "superbee"

fig, ax = plt.subplots(figsize=(14,8),dpi=80)
res, = ax.plot(x_vals,q_vals_square[0],lw=3.0,ls='-',color='r')
plt.xticks(size=15)
plt.yticks(size=15)
plt.xlabel("Increasing $x$ values $\longrightarrow$",size=18)
plt.ylabel("Increasing $q$ Values",size=18)
plt.title(limiter,size=20,fontweight='bold')
text = ax.text(8.5,0.97,r'Time: 0.00',fontsize=15,bbox={'facecolor':'pink','alpha':0.8,'pad':16})
plt.pause(2)

for n in range(len(t_vals)-1):
    for i in range(1,len(x_vals)-1):
        
        flux_left = flux_int(adv_vel,q_vals_square[n,i-2],q_vals_square[n,i-1],q_vals_square[n,i],dt,dx,limiter)
        flux_right = flux_int(adv_vel,q_vals_square[n,i-1],q_vals_square[n,i],q_vals_square[n,i+1],dt,dx,limiter)
        
        df = flux_right - flux_left
        
        q_vals_square[n+1,i] = q_vals_square[n,i] - (dt/dx)*df
        
    q_vals_square[n+1,0] = q_vals_square[n,-2]
    q_vals_square[n+1,-1] = q_vals_square[n,1]
        
    res.set_ydata(q_vals_square[n+1])
    text.set_text(r'Time: %.2f'%t_vals[n])
    fig.canvas.draw()
    fig.canvas.flush_events()
    
q_vals_square_superbee = q_vals_square

C:\Users\sheet\AppData\Local\Temp\ipykernel_10596\1902365474.py:9: RuntimeWarning: invalid value encountered in scalar divide
  r = (ql_1 - ql_2)/(q_0 - ql_1)
C:\Users\sheet\AppData\Local\Temp\ipykernel_10596\1902365474.py:9: RuntimeWarning: divide by zero encountered in scalar divide
  r = (ql_1 - ql_2)/(q_0 - ql_1)


In [28]:
limiter = "mc"
fig, ax = plt.subplots(figsize=(14,8),dpi=80)
res, = ax.plot(x_vals,q_vals_square[0],lw=3.0,ls='-',color='r')
plt.xticks(size=15)
plt.yticks(size=15)
plt.xlabel("Increasing $x$ values $\longrightarrow$",size=18)
plt.ylabel("Increasing $q$ Values",size=18)
plt.title(limiter,size=20,fontweight='bold')
text = ax.text(8.5,0.97,r'Time: 0.00',fontsize=15,bbox={'facecolor':'pink','alpha':0.8,'pad':16})
plt.pause(2)

for n in range(len(t_vals)-1):
    for i in range(1,len(x_vals)-1):
        
        flux_left = flux_int(adv_vel,q_vals_square[n,i-2],q_vals_square[n,i-1],q_vals_square[n,i],dt,dx,limiter)
        flux_right = flux_int(adv_vel,q_vals_square[n,i-1],q_vals_square[n,i],q_vals_square[n,i+1],dt,dx,limiter)
        
        df = flux_right - flux_left
        
        q_vals_square[n+1,i] = q_vals_square[n,i] - (dt/dx)*df
        
    q_vals_square[n+1,0] = q_vals_square[n,-2]
    q_vals_square[n+1,-1] = q_vals_square[n,1]
        
    res.set_ydata(q_vals_square[n+1])
    text.set_text(r'Time: %.2f'%t_vals[n])
    fig.canvas.draw()
    fig.canvas.flush_events()
    
q_vals_square_mc = q_vals_square

C:\Users\sheet\AppData\Local\Temp\ipykernel_10596\1902365474.py:9: RuntimeWarning: invalid value encountered in scalar divide
  r = (ql_1 - ql_2)/(q_0 - ql_1)
C:\Users\sheet\AppData\Local\Temp\ipykernel_10596\1902365474.py:9: RuntimeWarning: divide by zero encountered in scalar divide
  r = (ql_1 - ql_2)/(q_0 - ql_1)


In [29]:
limiter = "vanleer"

fig, ax = plt.subplots(figsize=(14,8),dpi=80)
res, = ax.plot(x_vals,q_vals_square[0],lw=3.0,ls='-',color='r')
plt.xticks(size=15)
plt.yticks(size=15)
plt.xlabel("Increasing $x$ values $\longrightarrow$",size=18)
plt.ylabel("Increasing $q$ Values",size=18)
plt.title(limiter,size=20,fontweight='bold')
text = ax.text(8.5,0.97,r'Time: 0.00',fontsize=15,bbox={'facecolor':'pink','alpha':0.8,'pad':16})
plt.pause(2)

for n in range(len(t_vals)-1):
    for i in range(1,len(x_vals)-1):
        
        flux_left = flux_int(adv_vel,q_vals_square[n,i-2],q_vals_square[n,i-1],q_vals_square[n,i],dt,dx,limiter)
        flux_right = flux_int(adv_vel,q_vals_square[n,i-1],q_vals_square[n,i],q_vals_square[n,i+1],dt,dx,limiter)
        
        df = flux_right - flux_left
        
        q_vals_square[n+1,i] = q_vals_square[n,i] - (dt/dx)*df
        
    q_vals_square[n+1,0] = q_vals_square[n,-2]
    q_vals_square[n+1,-1] = q_vals_square[n,1]
        
    res.set_ydata(q_vals_square[n+1])
    text.set_text(r'Time: %.2f'%t_vals[n])
    fig.canvas.draw()
    fig.canvas.flush_events()

q_vals_square_vanleer = q_vals_square

C:\Users\sheet\AppData\Local\Temp\ipykernel_10596\1902365474.py:9: RuntimeWarning: invalid value encountered in scalar divide
  r = (ql_1 - ql_2)/(q_0 - ql_1)
C:\Users\sheet\AppData\Local\Temp\ipykernel_10596\1902365474.py:9: RuntimeWarning: divide by zero encountered in scalar divide
  r = (ql_1 - ql_2)/(q_0 - ql_1)


In [30]:
#______Isothermal____________Shock________Tube
def tophat(x_val):
    if x_val > 0:
        return 1
    else:
        return 0
    
def flip_flop(vel_int):
    return 1 if vel_int >= 0 else -1


x_int = np.linspace(0,100,101,endpoint=True)

x_vals_2 = np.zeros(len(x_int)-1)

for i in range(len(x_vals_2)):
    x_vals_2[i] = (x_int[i]+x_int[i+1])/2
    
dx_2 = x_vals_2[1] - x_vals_2[0]

q_vals = np.zeros([3,len(x_vals_2)]) #Array for Primitive Variables
w_vals = np.zeros([2,len(x_vals_2)]) #Array for State (Conservative) Variables

sound_speed = np.ones(len(x_vals_2))
            
time = 0.0
eta_cfl = 0.5

#INITIAL CONDITION

for i in range(len(x_vals_2)):
    q_vals[0][i] = 2*tophat(50 - x_vals_2[i]) + 1
    q_vals[1][i] = 0 
    q_vals[2][i] = q_vals[0][i]*(sound_speed[i]**2)
             
    w_vals[0][i] = q_vals[0][i]
    w_vals[1][i] = q_vals[0][i]*q_vals[1][i]


fig, ax = plt.subplots(figsize=(14,8),dpi=80)
ax.plot(x_vals_2,q_vals[0],lw=3.0,ls='-.',color='k',label="Density - $\\rho$ at Time $t = 0.0$")
res0, = ax.plot(x_vals_2,q_vals[0],lw=3.0,ls='-',color='blue',label="Evolution of Density - $\\rho$")
res1, = ax.plot(x_vals_2,q_vals[1],lw=3.0,ls='-',color='red',label="Evolution of Velocity - $v$")
plt.xlabel("Increasing $x$ $\longrightarrow$",size=18)
plt.ylabel("Primitive Variables $\\rho$",size=18)
plt.title("Shock Tube Problem - Evolution",size=20,fontweight='bold')
plt.minorticks_on()
plt.xticks(size=18)
plt.yticks(size=18)
plt.legend(fontsize=15,facecolor='pink',loc="upper right")
plt.pause(2)

while time<=30.0:
    
    updated_w_array = w_vals
    updated_q_array = q_vals
    temp_w_array = w_vals
    
    v_vals_interface = np.zeros(len(x_int))
    flux_vals_interface = np.zeros([2,len(x_int)])
    
    a_array = np.zeros(len(x_vals_2))
    
    for i in range(len(x_vals_2)-1):
        a = max(abs(updated_q_array[1,i] - sound_speed[i]),
                abs(updated_q_array[1,i] + sound_speed[i]))
        
        a_array[i] = a
    
    dt_2 = eta_cfl * dx_2/max(a_array)
    
    for i in range(1,len(x_int)-1):
        v_vals_interface[i] = 0.5*((updated_w_array[1,i-1]/updated_w_array[0,i-1])
                                   + (updated_w_array[1,i]/updated_w_array[0,i]))
        
        theta = flip_flop(v_vals_interface[i])
        
        flux_vals_interface[0,i] = 0.5*v_vals_interface[i]*((1+theta)*updated_w_array[0,i-1] 
                                                          + (1-theta)*updated_w_array[0,i])
        flux_vals_interface[1,i] = 0.5*v_vals_interface[i]*((1+theta)*updated_w_array[1,i-1] 
                                                          + (1-theta)*updated_w_array[1,i])
    
    for i in range(1,len(x_vals_2)-1):
        temp_w_array[0,i] = updated_w_array[0,i] - ((dt_2/dx_2)*(flux_vals_interface[0,i+1]
                                                                 - flux_vals_interface[0,i]))
        temp_w_array[1,i] = updated_w_array[1,i] - ((dt_2/dx_2)*(flux_vals_interface[1,i+1]
                                                                 - flux_vals_interface[1,i]))
        
        updated_w_array[0,i] = temp_w_array[0,i]
    
    for i in range(1,len(x_vals_2)-1):
        updated_w_array[1,i] = temp_w_array[1,i] - 0.5*((sound_speed[i]**2)*(temp_w_array[0,i+1] 
                                                                             - temp_w_array[0,i-1])/dx_2)
    updated_w_array[0,0] = updated_w_array[0,1]
    updated_w_array[0,-1] = updated_w_array[0,-2]
    
    updated_w_array[1,0] = -updated_w_array[1,1]
    updated_w_array[1,-1] = -updated_w_array[1,-2]
    
    w_vals = updated_w_array

    q_vals[0] = w_vals[0]      
    q_vals[1] = w_vals[1]/w_vals[0]
    q_vals[2] = w_vals[0]*(sound_speed**2)
    
    time += dt_2
    
    res0.set_ydata(q_vals[0])
    res1.set_ydata(q_vals[1])
    fig.canvas.draw()
    fig.canvas.flush_events()